In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms 
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/kaggle/input/cbdata/Chess ID Public Data'  # replace with the path to your data
image_datasets = {
    'train': datasets.ImageFolder(data_dir + '/output_train', data_transforms['train']),
    'test': datasets.ImageFolder(data_dir + '/output_test', data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=4)
}

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(image_datasets['train'].classes))  # Adjust for number of chess piece classes
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 5

In [9]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    # Training phase
    model.train()
    running_loss = 0.0
    running_corrects = 0
    progress_bar = tqdm(dataloaders['train'], desc=f'Training Epoch {epoch+1}')
    
    for inputs, labels in progress_bar:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        progress_bar.set_postfix(loss=running_loss / len(image_datasets['train']),
                                 acc=running_corrects.double() / len(image_datasets['train']))
    
    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects.double() / len(image_datasets['train'])
    
    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    
    model.eval()
    test_loss = 0.0
    test_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloaders['test'], desc=f'Testing Epoch {epoch+1}'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            _, preds = torch.max(outputs, 1)
            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    test_loss = test_loss / len(image_datasets['test'])
    test_acc = test_corrects.double() / len(image_datasets['test'])
    
    print(f'Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}\n')

print('Training complete!')
model.save('chess.pt')

Epoch 1/5
----------


Training Epoch 1: 100%|██████████| 324/324 [02:32<00:00,  2.13it/s, acc=tensor(0.8424, dtype=torch.float64), loss=0.49] 


Train Loss: 0.4898 Acc: 0.8424


Testing Epoch 1: 100%|██████████| 114/114 [00:15<00:00,  7.28it/s]


Test Loss: 0.0137 Test Acc: 0.9978

Epoch 2/5
----------


...

Training Epoch 5: 100%|██████████| 324/324 [02:34<00:00,  2.10it/s, acc=tensor(0.9565, dtype=torch.float64), loss=0.138] 


Train Loss: 0.1377 Acc: 0.9565


Testing Epoch 5: 100%|██████████| 114/114 [00:15<00:00,  7.57it/s]

Test Loss: 0.0105 Test Acc: 0.9973

Training complete!


AttributeError: 'ResNet' object has no attribute 'save'

In [12]:
torch.save(model.state_dict(), 'chess.pt')